# [Bonus Tasks](svm_segm.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [3]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [4]:
def create_data_matrix(img) :
    patch = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    num_patch = patch.shape[0] * patch.shape[1]
    X = patch.reshape(num_patch, -1)
    
    return np.array(X)

**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [47]:
def create_gt_labels_vector(img) :
    patch = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    
    rows, cols = patch.shape[:2]
    labels = np.empty(rows * cols, dtype = int)
    

    a = 0

    for i in range(rows) :
        for j in range(cols) :
            patches = patch[i, j]

            if np.mean(patch) > 0.5 :
                labels[a] = 1
            
            elif np.mean(patch) == 0 :
                labels[a] = -1
            
            else :
                labels[a] = 0

            a += 1

    return np.array(a)

**[Task 1.3.]()** Create the SVM classifier:

In [48]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [49]:
dna33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna44 = plt.imread('data/NIH3T3/im/dna-44.png')

a3 = create_data_matrix(dna33)
a4 = create_data_matrix(dna44)

a3.dtype
print(a3.shape)


(1344, 1024)


**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [50]:

g3 = plt.imread('data/NIH3T3/gt/33.png')
g4 = plt.imread('data/NIH3T3/gt/44.png')

b3 = create_gt_labels_vector(g3)
b4 = create_gt_labels_vector(g4)

print(b3.shape)
b3.dtype

()


dtype('int64')

**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [51]:
a_valid = np.concatenate([a3[(b3 == 1) | (b3 == -1)], a4[(b4 == 1) | (b4 == -1)]])
b_valid = np.concatenate([b3[(b3 == 1) | (b3 == -1)], b4[(b4 == 1) | (b4 == -1)]])

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [52]:
clf.fit(a_valid, b_valid) 
pd.read_csv()

ValueError: Found array with dim 3. StandardScaler expected <= 2.

**[Task 1.4.]()** Implement the function `predict_image`:

In [32]:
def predict_image(img) :

    X = create_data_matrix(img)
    prediction = clf.predict(X)
    result_falses = np.zeros(img.shape, dtype=bool)
    
    # view from the result in windows (if changed here, also changed in results)
    patch_height, patch_width = patch_sizes
    step = patch_width // 2
    patches = skimage.util.view_as_windows(result_falses, window_shape = patch_sizes, step = step)
    t = np.full(patch_sizes, True, dtype=bool)

    for i in range(rows) :
        for n in range(cols) :
            # convert the patch index into the corresponding labels index in the vector
            num = i * patches.shape[1] + n
            # if the label in the prediction is 1, change the patch to all True
            if prediction[num] == 1 :       
                patches[i, n] = t
    

    return result

Test your implementation:

In [34]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

print(seg)

plt.imshow(seg, vmin=0, vmax=1, cmap='gray')


plt.figure()
plt.imshow(blend_result(img, seg))

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

**[Task 1.5.]()** Perform batch processing:

In [35]:
seq = [28, 29, 33, 44, 46, 49]

all = []

for i in seq :
    img = plt.imread(f'data/NIH3T3/im/dna-{i}.png')
    ground = plt.imread(f'data/NIH3T3/gt/{i}.png')

    binground = ground > 0.5
    binimg = predict_image(img)
    
    dice = 2 * (binimg * binground).sum() / (binimg.sum() + binground.sum())

    all.append(dice)

    print('image: ' + str(i) + ', Dice: ' + str(round(dice, 2)))

print()
print('Average Dice: ' + str(round(sum(all)/len(all), 2)))

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.